In [1]:
%%writefile mongo_extract.py
'''
This script converts the raw html data in mongo db bills.pages and puts the relevant data
in dictionary format in bill.bill_details
'''

from pymongo import MongoClient
import bson.json_util
import copy
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

import codecs
import json

def write_json_file(obj, path):
    '''Dump an object and write it out as json to a file'''
    f = codecs.open(path, 'a', 'utf-8')
    json_record = json.dumps(obj, ensure_ascii = False)
    f.write(json_record + '\n')
    f.close
    
    
def soup_to_mongo(soup, collection_name):
    '''Scrapes a single soup object retrived from a table for relevant bill information. 
    It creates a dictionary for each row in the soup object and loads it into 
    mongo database collection_name.  Note: Selenium has been timing out when trying to 
    retrieve cosponsor information. Commented these lines out for the time being.
    
    Parameters: soup object
                mongo database collection name
    
    Returns:    None
    '''
    # table of bills are in ol class
    div = soup.find('div', {'class':'search-column-main'})
    table = div.find('ol')

    # iterate though each li class expanded to get rows
    rows = table.find_all('li', {'class':'expanded'})
    print('\tThere are {} rows to iterate through on this pass.'.format(len(rows)))
    

    # store each row as key-value pair in a dictionary
    empty_row = {'leg_id': None, 
                'leg_type': None,
                'leg_url': None,
                'intro_date': None,
                'congress_id': None,
                'desc': None,
                'sponsor': None, 
                'sponsor_party': None, 
                'sponsor_state': None,
                'sponsor_district': None,  #senators don't have districts
                'num_of_cosponsors': None,
                'cosponsors_url': None,
                'cosponsors': None,  #requires navigation to another url and extracting names from table
                'committee': None, 
                'bill_status': None,
                'body': None   #requires navigation to another url
                }


    # for eyeball examination/debugging
    # columns = rows[9].find_all('a')
    # columns

#     all_rows = []
    i = 0

    # iterate through each of the 'rows' to fill in the 'columns'
    for row in rows:
        new_row = copy.copy(empty_row)

        # parse items within 'a' tag
        columns = row.find_all('a')
        if columns[0].text.strip() != '':
            new_row['leg_id'] = columns[0].text.strip().replace('.', ' ')
#             print('--------------')
#             print(new_row['leg_id'])

 
        # reserved bill numbers will not have the information below
        if (len(columns) > 2):    
            if columns[0]['href'].strip() != '':
                new_row['leg_url'] = columns[0]['href'].strip()
            if columns[2].text.strip() != '':
                new_row['num_of_cosponsors'] = columns[2].text.strip()
                if new_row['num_of_cosponsors'] != '0':
                    new_row['cosponsors_url'] = columns[2]['href']
                    # call function to get cosponsors table from url
                    # commented this line out for timout errors
    #                 new_row['cosponsors'] = get_cosponsors(cosponsors_url)



            # party, state, and district (for house reps) need to be stripped out of sponsor info
            # Amendments have the rep in columns[2]
            
#         if columns[1].text.strip() != '':
            for c in range(len(columns)):
                if '[' in columns[c].text.strip():
                    rep = columns[c].text.strip()
                    new_row['sponsor'] = rep.rsplit('[', 1)[0][:-1]
                    party_dist = rep.rsplit('[', 1)[1][: -1]
                    party_dist_split = party_dist.split('-')
                    new_row['sponsor_state'] = party_dist_split[0]
                    new_row['sponsor_party'] = party_dist_split[1]
                    if len(party_dist_split) == 3:
                        new_row['sponsor_district'] = party_dist_split[2]

            # parse items within 'span' tag
            columns = row.find_all('span')
            if columns[0].text != '':
                new_row['leg_type'] = columns[0].text.strip()
            if columns[1].text.strip().split()[2] != '':
                new_row['congress_id'] = columns[1].text.strip().split()[2]
            if columns[2].text != '':
                new_row['desc'] = columns[2].text
            if len(columns) > 4:
                if ('Committee' in columns[4].text):
                    new_row['committee'] = columns[4].text.strip()[12:]
                
                
            # date was a little tricky... actually, a few things were after some trys
            dt = columns[3].text.strip().split()
            if '(Introduced' in dt:
                new_row['intro_date'] = dt[dt.index('(Introduced') + 1][:-1]

            # parse items within p tag for bill_status
            # Amendments will not have a bill status
            if 'Amdt' not in new_row['leg_id']:
                columns = row.find_all('p')
#                 print(columns)
                if columns[0].text.strip()[25:] != '':
                    new_row['bill_status'] = columns[0].text.strip()[25:]

#         all_rows.append(new_row)
        collection_name.insert_one(new_row)
        
        i += 1
        if i%50 == 0:
            print('\t\t{:.2f}% complete'.format(100 * i / len(rows)))
        
#     return all_rows


# this function really slows the data gathering process down... AND it's been timing out
def get_cosponsors(site_url):
    '''Scrapes site_url for list of cosponsors.
    
    Parameters: url
    
    Returns: list of dictionaries with cosponsor information 
    '''
    url = site_url
#     print(url)

    # send GET request using selenium (sites in javascript)
    option = webdriver.ChromeOptions()
    option.add_argument(' - incognito')
    option.add_argument('--headless')

    browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver', chrome_options=option)

    req = requests.get(url)
    
    if req.status_code == 200:
        soup = BeautifulSoup(req.content, 'lxml')
        div = soup.find('div', {'id':'main'})
        table = div.find('table')
        rows = table.find_all('a')

        empty_row = {
            'cosponsor_name': None, 
            'cosponsor_party': None, 
            'cosponsor_state': None,
            'cosponsor_district': None
        }

        all_rows = []

        for row in rows:
            new_row = copy.copy(empty_row)
            rep = row.text.strip()

            new_row['cosponsor_name'] = rep.rsplit('[', 1)[0][:-1]
            party_dist = rep.rsplit('[', 1)[1][: -1]
            party_dist_split = party_dist.split('-')
            new_row['cosponsor_state'] = party_dist_split[0]
            new_row['cosponsor_party'] = party_dist_split[1]
            if len(party_dist_split) == 3:
                new_row['cosponsor_district'] = party_dist_split[2][:-1]

            all_rows.append(new_row)

        return all_rows
    
    else:
        output = {'url': url, 'error': req.status_code}
        outpath = '../data/logs/cosponsor_errors.jsonl'
        write_json_file(outpath, output)

   

  
   

if __name__ == '__main__':
    # Set up Mongo for raw data and prettified data
    client = MongoClient() # defaults to localhost
    db = client.bills

    # collection 'pages' is where the raw html data is at
    # collection 'bill_details' is where the prettified data will go 
    pages = db.pages

    bill_details = db.bill_details

    num_items = pages.find().count()
    print('There are {} items in this collection.'.format(num_items))
    
    items = pages.find()

    x = 0
    
    for i in items:
        soup = BeautifulSoup(i['lxml'], 'lxml')
        
        # keep track of what's in the queue before parsing it
        mongo_id = str(i['_id'])
        mongo_log = '../data/logs/mongo_updates.jsonl'
        write_json_file(mongo_id, mongo_log)

        soup_to_mongo(soup, bill_details)

        x += 1
        if x%20 == 0:
            print('Overall, {:.2f}% complete.'.format(100 * x / num_items))

Overwriting mongo_extract.py


In [109]:
# exploration
from pymongo import MongoClient
import bson.json_util
import copy
import requests
from bs4 import BeautifulSoup
from bson.objectid import ObjectId
import pprint

client = MongoClient() # defaults to localhost
db = client.bills

# collection 'pages' is where the raw data is at
# collection 'bill_details' is where the prettified data will go 
pages = db.pages

bill_details = db.bill_details

In [112]:
num_items = pages.find().count()
print('There are {} items in this collection.'.format(num_items))

item = db.pages.find({'_id': ObjectId('5c062533cd68d162a94bbc3c')})

soup = BeautifulSoup(item[0]['lxml'], 'lxml')
soup_to_mongo(soup, bill_details)

In [113]:
bill_details.find().count()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


253000

In [135]:
hr7211 = bill_details.find({'leg_id': 'H R 1222', 'congress_id': '115th'})
for h in hr7211:
    pprint.pprint(h)


{'_id': ObjectId('5c11d654cd68d16918e5a60e'),
 'bill_status': 'Passed House',
 'body': None,
 'committee': 'House - Energy and Commerce | Senate - Health, Education, '
              'Labor, and Pensions',
 'congress_id': '115th',
 'cosponsors': None,
 'cosponsors_url': 'https://www.congress.gov/bill/115th-congress/house-bill/1222/cosponsors?r=5990&overview=closed#tabs',
 'desc': 'Congenital Heart Futures Reauthorization Act of 2017',
 'intro_date': '02/27/2017',
 'leg_id': 'H R 1222',
 'leg_type': 'BILL',
 'leg_url': 'https://www.congress.gov/bill/115th-congress/house-bill/1222?r=5990',
 'num_of_cosponsors': '28',
 'sponsor': 'Rep. Bilirakis, Gus M.',
 'sponsor_district': '12',
 'sponsor_party': 'FL',
 'sponsor_state': 'R'}


In [129]:
import os
# for f in os.listdir('../data'):
#     if f.startswith('bill_text'):
#         print(f)

In [124]:
def read_jsonl_file(path):
    '''turn a jsonl file into an array of objects'''
    arr = []
    f = codecs.open(path, 'r', 'utf-8')
    for line in f:
        record = json.loads(line.rstrip('\n|\r'))
        arr.append(record)
    return arr

In [125]:
bt_2018 = read_jsonl_file('../data/bill_texts_2018.jsonl')

In [155]:
len(bt_2018)

228

In [139]:
bt_2018[10]
b_issue = bt_2018[10]['issue']
c_id = bt_2018[10]['congress_id'][:5]
b_text = bt_2018[10]['bill_text']


In [154]:
item = bill_details.find({'leg_id': b_issue, 'congress_id': c_id})
for i in item:
    pprint.pprint(i)

{'_id': ObjectId('5c11d654cd68d16918e5a60e'),
 'bill_status': 'Passed House',
 'body': '[Congressional Bills 115th Congress] [From the U.S. Government '
         'Publishing Office] [H.R. 1222 Reported in Senate (RS)] <DOC> '
         'Calendar No. 555 115th CONGRESS 2d Session H. R. 1222 '
         '_______________________________________________________________________ '
         'IN THE SENATE OF THE UNITED STATES February 27, 2018 Received; read '
         'twice and referred to the Committee on Health, Education, Labor, and '
         'Pensions August 15, 2018 Reported by Mr. Alexander, with an '
         'amendment [Strike out all after the enacting clause and insert the '
         'part printed in italic] '
         '_______________________________________________________________________ '
         'AN ACT To amend the Public Health Service Act to coordinate Federal '
         'congenital heart disease research efforts and to improve public '
         'education and awareness of

In [153]:
bill_details.update({'leg_id': b_issue, 'congress_id': c_id}, {'$set':{'body':b_text}})

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}